# Password Strength Prediction using Machine Learning & NLP


### Objective
The objective of this project is to build a machine learning system that predicts the strength of a password using character-level NLP techniques and security-inspired engineered features. The model classifies passwords into multiple strength levels ranging from *Very Weak* to *Very Strong*, with a focus on identifying weak and risky passwords accurately.

### Business / Security Problem
Weak and predictable passwords are a major cause of security breaches and account compromise. Many existing password strength meters rely on simple heuristics such as minimum length or the presence of symbols, which often fail to reflect real-world password patterns.

This project addresses the problem of **automated password strength estimation** by learning patterns from large-scale real-world password data. The goal is not to crack passwords, but to assess their relative strength and highlight risky choices in a responsible and ethical manner.

### Problem Type
- Supervised Machine Learning  
- Multi-class Classification  
- Input: Raw password strings  
- Output: Password strength category  


### Strength Classes
The password strength prediction task is formulated as a multi-class classification problem with the following categories:

- Very Weak  
- Weak  
- Medium  
- Strong  
- Very Strong  

This setup provides finer granularity compared to binary or three-class systems and better reflects real-world password quality differences.


### Datasets Used

#### Primary Dataset (Main Project Dataset)
**PWLDS – Password Weakness and Level Dataset**

- Large-scale dataset containing real-world passwords
- Used for:
  - Feature engineering
  - Model training and evaluation
  - Error analysis
  - Final conclusions

#### Secondary Dataset (Baseline Comparison Only)
**Password Strength Classifier Dataset (Kaggle)**

- Smaller, lower-granularity dataset
- Used only for:
  - Training a simple baseline model
  - External comparison
  - Strengthening evaluation credibility

> The secondary dataset is not used for feature design, hyperparameter tuning, or final model conclusions.


### Evaluation Focus
Model evaluation emphasizes:
- Overall performance using macro F1-score
- Recall on **Very Weak** and **Weak** password classes, as misclassifying weak passwords as strong has higher security risk


### Ethical & Responsible Use
This project uses publicly available leaked-password datasets strictly for educational and analytical purposes. No real user-entered passwords are collected, logged, or stored. The resulting models are intended for research and learning, not for real-world password validation systems.


### Project Scope & Limitations
- The model estimates relative password strength, not exact cracking time
- It does not simulate real password attacks
- Results are dataset-dependent and may not generalize to all user populations


## Notebook Roadmap
The notebook is organized into clearly defined phases:

- Phase 1: Data Loading & Understanding  
- Phase 2: Feature Engineering  
- Phase 3: Baselines & Ablation Study  
- Phase 4: Tree-Based Models  
- Phase 5: Neural Network Model  
- Phase 6: External Baseline Comparison  
- Phase 7: Security Interpretation & Research Context  
- Phase 8: Final Results & Conclusions  

# ===============================
# Phase 1: Data Loading & Understanding
# ===============================

In [1]:
import pandas as pd

# Load only required columns to keep memory usage under control
pwlds = pd.read_csv("pwlds_full.csv")

pwlds.head()

,Password,Strength_Level
0,7hqwv,0
1,cjml,0
2,asuy,0
3,kcyth,0
4,whcq,0


In [2]:
# Standardize column names
pwlds = pwlds.rename(columns={
    "Password": "password",
    "Strength_Level": "strength"
})

pwlds.head()

,password,strength
0,7hqwv,0
1,cjml,0
2,asuy,0
3,kcyth,0
4,whcq,0


In [3]:
pwlds.shape

(10000470, 2)

In [4]:
pwlds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000470 entries, 0 to 10000469
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   password  object
 1   strength  int64 
dtypes: int64(1), object(1)
memory usage: 152.6+ MB


In [5]:
# Check for missing values
pwlds.isna().sum()

password    4
strength    0
dtype: int64

In [6]:
pwlds = pwlds.dropna(subset=["password"])
pwlds.isna().sum()

password    0
strength    0
dtype: int64

In [7]:
## Strength Level Distribution

In [8]:
pwlds["strength"].value_counts()

strength
3    2000382
0    2000039
2    2000024
1    2000021
4    2000000
Name: count, dtype: int64

In [9]:
## Password Length Distribution

In [10]:
# Compute password length
pwlds["password_length"] = pwlds["password"].str.len()

pwlds["password"].describe()

count       10000466
unique       8635302
top       abolisherL
freq               4
Name: password, dtype: object

## Phase 1 Checkpoint

At this stage, we have:
- Successfully loaded a large-scale password dataset
- Understood dataset size and schema
- Identified class distribution and potential imbalance
- Observed basic password characteristics

In the next phase, we will focus on transforming raw passwords into
meaningful numerical features for machine learning models.

### Duplicate Password Analysis

Real-world password datasets often contain duplicate passwords because
many users choose the same or similar passwords. Before modeling, we
analyze duplicate password occurrences and decide how to handle them.

At this stage, we focus only on understanding duplicates, not removing
them prematurely.


In [11]:
# Count duplicate password entries
total_rows = len(pwlds)
unique_passwords = pwlds["password"].nunique()

total_rows, unique_passwords, total_rows - unique_passwords

(10000466, 8635302, 1365164)

### Duplicate Passwords with Conflicting Labels

A critical check is whether the same password appears with different
strength labels. If this happens, we must resolve label inconsistency
before modeling.


In [12]:
# Check if the same password appears with multiple strength labels
label_variation = (
    pwlds.groupby("password")["strength"]
    .nunique()
    .sort_values(ascending=False)
)

label_variation.head()

password
acerbityT|    2
abrogator^    2
Achyrodesl    2
abrogator[    2
acentrouso    2
Name: strength, dtype: int64

In [13]:
# Number of passwords mapped to more than one strength class
(label_variation > 1).sum()

37095

### Interpretation

- Duplicate passwords are expected in real-world datasets and reflect
  common user behavior.
- If a password appears with multiple strength labels, this indicates
  label noise or ambiguity.
- We will decide how to handle such cases explicitly before modeling
  to avoid confusing the learning algorithms.


### Decision on Duplicate Passwords

- Duplicate passwords with the **same strength label** are retained, as
  they reflect real-world frequency information.
- Passwords that appear with **conflicting strength labels** introduce
  ambiguity and will be handled carefully in the next step.

The final decision (retain, deduplicate, or resolve conflicts) will be
made explicitly and documented before feature engineering.


### Resolving Duplicate Passwords with Conflicting Labels

Some passwords may appear multiple times in the dataset with different
strength labels. This introduces label ambiguity, which can negatively
affect supervised learning models.

To ensure label consistency, we explicitly identify and handle such cases
before proceeding to feature engineering.


In [14]:
# Identify passwords that map to more than one strength label
conflicting_passwords = (
    pwlds.groupby("password")["strength"]
    .nunique()
    .reset_index(name="label_count")
)

conflicting_passwords = conflicting_passwords[
    conflicting_passwords["label_count"] > 1
]

len(conflicting_passwords)

37095

In [15]:
# Store row count before cleaning
rows_before = len(pwlds)

# Remove passwords with conflicting strength labels
pwlds = pwlds[~pwlds["password"].isin(conflicting_passwords["password"])]

rows_after = len(pwlds)

rows_before, rows_after, rows_before - rows_after

(10000466, 9889407, 111059)

### Post-Cleaning Validation

After resolving duplicate label conflicts, we verify that:
- Each password maps to exactly one strength label
- No missing values remain in critical columns


In [16]:
# Confirm no password maps to multiple strength labels
pwlds.groupby("password")["strength"].nunique().max()

1

In [17]:
# Final missing value check
pwlds.isna().sum()


password           0
strength           0
password_length    0
dtype: int64

### Phase 1 Summary

In this phase, we:
- Loaded a large-scale real-world password dataset
- Standardized column names
- Removed records with missing passwords
- Analyzed and handled duplicate passwords
- Eliminated label ambiguity to ensure clean supervision

The dataset is now consistent and ready for feature engineering.

# ===============================
# Phase 2: Feature Engineering & Feature-Level EDA
# ===============================

## Purpose
Transform raw password strings into meaningful numerical features that
capture structural complexity, character composition, and information
content relevant to password strength.

## What We Do
- Ensure raw password data remains intact
- Engineer structured security features
- Add an information-theoretic entropy feature
- Generate character-level NLP features using TF-IDF (memory-safe)

## Expected Output
- Clean, interpretable engineered features
- Scalable NLP feature representation
- Feature sets ready for modeling and ablation

## 2.0 Ensure Correct Data Type for Passwords

Before applying string-based feature engineering, we explicitly cast the
password column to string type to avoid issues caused by mixed or inferred
data types in large CSV files.

In [18]:
# Ensure passwords are treated as strings (do NOT overwrite the column later)
pwlds["password"] = pwlds["password"].astype(str)

### Remove Invalid Password Artifacts

After enforcing string type, some invalid artifacts (e.g., 'nan', 'None',
or extremely short values) may exist due to casting. These entries do not
represent real passwords and are removed before feature computation.


In [19]:
# Remove invalid password artifacts introduced by string casting
pwlds = pwlds[pwlds["password"].str.len() > 1]
pwlds = pwlds[~pwlds["password"].isin(["nan", "None", "null"])]

pwlds.shape

(9889333, 3)

### Structured Security Features

We engineer interpretable, security-inspired features that capture
basic password characteristics such as length, character composition,
and repetition patterns.


In [20]:
import numpy as np
import re

In [21]:
pwlds["password_length"] = pwlds["password"].str.len()

In [22]:
# Count different character types
pwlds["upper_count"] = pwlds["password"].str.count(r"[A-Z]")
pwlds["lower_count"] = pwlds["password"].str.count(r"[a-z]")
pwlds["digit_count"] = pwlds["password"].str.count(r"[0-9]")
pwlds["symbol_count"] = pwlds["password"].str.count(r"[^A-Za-z0-9]")


### Character Ratios

Raw counts can be misleading for passwords of different lengths.
We therefore compute ratios to normalize character composition.


In [23]:
# Avoid division by zero by relying on cleaned data
pwlds["upper_ratio"] = pwlds["upper_count"] / pwlds["password_length"]
pwlds["digit_ratio"] = pwlds["digit_count"] / pwlds["password_length"]
pwlds["symbol_ratio"] = pwlds["symbol_count"] / pwlds["password_length"]

### Repetition Ratio

Repetition reduces effective password strength. We capture this by
measuring how many characters are repeated within a password.


In [24]:
def repetition_ratio(password: str) -> float:
    if len(password) == 0:
        return 0.0
    return 1 - (len(set(password)) / len(password))

pwlds["repetition_ratio"] = pwlds["password"].apply(repetition_ratio)

### Feature Sanity Check

Before proceeding further, we perform a quick sanity check to ensure
engineered features behave as expected.


In [25]:
pwlds[
    [
        "password_length",
        "upper_count",
        "digit_count",
        "symbol_count",
        "upper_ratio",
        "digit_ratio",
        "symbol_ratio",
        "repetition_ratio"
    ]
].describe()

,password_length,upper_count,digit_count,symbol_count,upper_ratio,digit_ratio,symbol_ratio,repetition_ratio
count,9.889333e+06,9.889333e+06,9.889333e+06,9.889333e+06,9.889333e+06,9.889333e+06,9.889333e+06,9.889333e+06
mean,1.235436e+01,1.927345e+00,9.765136e-01,2.163861e+00,1.093403e-01,6.912703e-02,1.140687e-01,1.419803e-01
std,6.866397e+00,2.815208e+00,1.459941e+00,3.426673e+00,1.271187e-01,1.029088e-01,1.477168e-01,1.226284e-01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.100000e+01,1.000000e+00,0.000000e+00,1.000000e+00,8.333333e-02,0.000000e+00,8.333333e-02,1.176471e-01
75%,1.300000e+01,2.000000e+00,1.000000e+00,2.000000e+00,1.923077e-01,1.000000e-01,2.000000e-01,2.000000e-01
max,3.200000e+01,2.100000e+01,1.200000e+01,2.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,8.888889e-01


## 2.5 Information-Theoretic Feature: Normalized Entropy

Entropy measures character diversity within a password. We use normalized
Shannon entropy to make the metric comparable across different password
lengths.

Entropy is treated as a supporting feature rather than a standalone
strength estimator.

In [26]:
from collections import Counter
import math

def normalized_entropy(password: str) -> float:
    if len(password) <= 1:
        return 0.0

    counts = Counter(password)
    length = len(password)

    entropy = 0.0
    for count in counts.values():
        p = count / length
        entropy -= p * math.log2(p)

    # Normalize by maximum possible entropy for given length
    max_entropy = math.log2(length)
    return entropy / max_entropy if max_entropy > 0 else 0.0


In [27]:
pwlds["entropy"] = pwlds["password"].apply(normalized_entropy)

In [28]:
pwlds["entropy"].describe()

count    9.889333e+06
mean     9.085718e-01
std      9.327933e-02
min      0.000000e+00
25%      8.605285e-01
50%      9.397940e-01
75%      1.000000e+00
max      1.000000e+00
Name: entropy, dtype: float64

In [29]:
pwlds.groupby("strength")["entropy"].mean()

strength
0    0.943730
1    0.817784
2    0.899928
3    0.932397
4    0.949341
Name: entropy, dtype: float64

In [30]:
pwlds.groupby("strength")["entropy"].describe()

,count,mean,std,min,25%,50%,75%,max
strength,,,,,,,,
0,1998746.0,0.943730,0.106114,0.000000,1.000000,1.000000,1.000000,1.0
1,1998009.0,0.817784,0.113934,0.000000,0.737214,0.796658,0.916667,1.0
2,1945205.0,0.899928,0.057718,0.546490,0.860529,0.907019,0.947443,1.0
3,1947373.0,0.932397,0.055400,0.555834,0.887436,0.939794,1.000000,1.0
4,2000000.0,0.949341,0.028583,0.738742,0.931092,0.950440,0.971218,1.0


### Entropy Interpretation

Normalized entropy captures character diversity but does not encode
password length or real-world guessing difficulty. Its behavior in this
dataset confirms that entropy alone is insufficient, motivating the use
of hybrid feature sets.

### NLP Features: Character-Level TF-IDF

To capture local character patterns (e.g., substrings, keyboard sequences),
we use character-level TF-IDF. Due to dataset size, vocabulary learning is
performed on a representative stratified sample.


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Stratified Sampling for TF-IDF Vocabulary Learning

TF-IDF vocabulary learning is memory-intensive. We therefore fit the
vectorizer on a stratified sample to balance representativeness and
computational feasibility.


In [32]:
sample_size = 300_000

pwlds_sample = (
    pwlds
    .groupby("strength", group_keys=False)
    .apply(lambda x: x.sample(
        n=min(len(x), sample_size // pwlds["strength"].nunique()),
        random_state=42
    ))
)

pwlds_sample.shape

C:\Users\user\AppData\Local\Temp\ipykernel_12680\1883506281.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


(300000, 12)

In [33]:
tfidf_vectorizer = TfidfVectorizer(
    analyzer="char",
    ngram_range=(2, 5),
    max_features=5000,
    min_df=2
)

In [34]:
X_tfidf_sample = tfidf_vectorizer.fit_transform(pwlds_sample["password"])
X_tfidf_sample.shape

(300000, 5000)

## Phase 2 Summary

In this phase, we:
- Preserved the raw password column without corruption
- Engineered interpretable security features
- Added normalized entropy with documented limitations
- Learned character-level TF-IDF features using a memory-safe strategy

The dataset is now fully prepared for baseline modeling and ablation
studies in Phase 3.

## ===============================
## Phase 3: Baselines & Ablation Study
## ===============================

### Purpose
Establish strong baselines and quantify the contribution of different
feature sets through an ablation study.

### What We Do
- Define clear feature sets
- Train simple baseline models
- Compare performance across feature sets
- Focus evaluation on weak-password recall

### Expected Output
- Baseline metrics
- Ablation comparison table
- Clear justification for feature choices


### Feature Set Definitions

We evaluate three feature configurations:
1. Engineered features only
2. TF-IDF features only (sample-based)
3. Combined engineered + TF-IDF features

This allows us to quantify the contribution of each feature type.


In [35]:
# Engineered feature columns (from Phase 2)
engineered_features = [
    "password_length",
    "upper_count",
    "lower_count",
    "digit_count",
    "symbol_count",
    "upper_ratio",
    "digit_ratio",
    "symbol_ratio",
    "repetition_ratio",
    "entropy",
]

X_eng = pwlds_sample[engineered_features]
y_sample = pwlds_sample["strength"]

### Train / Validation Split

We split the sampled data into training and validation sets using
stratification to preserve class distribution.


In [36]:
from sklearn.model_selection import train_test_split

X_eng_train, X_eng_val, y_train, y_val = train_test_split(
    X_eng,
    y_sample,
    test_size=0.2,
    random_state=42,
    stratify=y_sample
)

### Logistic Regression Baseline (Engineered Features)

We start with a simple, interpretable linear model to establish a baseline
and assess whether engineered features alone carry useful signal.


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

logreg_eng = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight="balanced"
)

logreg_eng.fit(X_eng_train, y_train)

y_pred_eng = logreg_eng.predict(X_eng_val)

print(classification_report(y_val, y_pred_eng))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     12000
           1       0.86      0.91      0.89     12000
           2       0.73      0.91      0.81     12000
           3       0.91      0.63      0.74     12000
           4       1.00      1.00      1.00     12000

    accuracy                           0.89     60000
   macro avg       0.90      0.89      0.89     60000
weighted avg       0.90      0.89      0.89     60000



In [38]:
f1_eng = f1_score(y_val, y_pred_eng, average="macro")
f1_eng

0.8876270855248828

## 3.4 Logistic Regression Baseline (TF-IDF Only)

Next, we evaluate a pure NLP baseline using character-level TF-IDF
features to understand how much signal is captured without engineered
features.


In [39]:
# Align TF-IDF rows with the sampled dataframe
X_tfidf = X_tfidf_sample


In [40]:
X_tfidf_train, X_tfidf_val, _, _ = train_test_split(
    X_tfidf,
    y_sample,
    test_size=0.2,
    random_state=42,
    stratify=y_sample
)


In [41]:
logreg_tfidf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight="balanced"
)

logreg_tfidf.fit(X_tfidf_train, y_train)

y_pred_tfidf = logreg_tfidf.predict(X_tfidf_val)

print(classification_report(y_val, y_pred_tfidf))


              precision    recall  f1-score   support

           0       0.96      0.98      0.97     12000
           1       0.98      0.96      0.97     12000
           2       0.87      0.93      0.90     12000
           3       0.83      0.69      0.75     12000
           4       0.88      0.98      0.93     12000

    accuracy                           0.91     60000
   macro avg       0.90      0.91      0.90     60000
weighted avg       0.90      0.91      0.90     60000



In [42]:
f1_tfidf = f1_score(y_val, y_pred_tfidf, average="macro")
f1_tfidf

0.90329384469112

### Combined Feature Baseline

We combine engineered features with TF-IDF features to test whether
structural and NLP-based signals complement each other.


In [43]:
from scipy.sparse import hstack

X_combined = hstack([X_eng, X_tfidf])


In [44]:
X_comb_train, X_comb_val, _, _ = train_test_split(
    X_combined,
    y_sample,
    test_size=0.2,
    random_state=42,
    stratify=y_sample
)


In [45]:
logreg_comb = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight="balanced"
)

logreg_comb.fit(X_comb_train, y_train)

y_pred_comb = logreg_comb.predict(X_comb_val)

print(classification_report(y_val, y_pred_comb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12000
           1       0.99      0.99      0.99     12000
           2       0.91      0.96      0.93     12000
           3       0.96      0.90      0.92     12000
           4       1.00      1.00      1.00     12000

    accuracy                           0.97     60000
   macro avg       0.97      0.97      0.97     60000
weighted avg       0.97      0.97      0.97     60000



In [46]:
f1_comb = f1_score(y_val, y_pred_comb, average="macro")
f1_comb

0.9693982611780205

### Ablation Results Summary

We summarize macro F1-scores across feature sets to quantify their impact.


In [47]:
import pandas as pd

ablation_results = pd.DataFrame({
    "Feature_Set": [
        "Engineered Features Only",
        "TF-IDF Only",
        "Engineered + TF-IDF"
    ],
    "Macro_F1": [
        f1_eng,
        f1_tfidf,
        f1_comb
    ]
})

ablation_results

,Feature_Set,Macro_F1
0,Engineered Features Only,0.887627
1,TF-IDF Only,0.903294
2,Engineered + TF-IDF,0.969398


## Phase 3 Summary

In this phase, we:
- Established logistic regression baselines
- Performed a controlled ablation study
- Quantified the contribution of engineered and NLP features
- Identified the most effective feature combination

These results guide model selection for more powerful non-linear models
in the next phase.

## ===============================
## Phase 4: Tree-Based Models (Main Results)
## ===============================

## Purpose
Tree-based models are used to capture non-linear relationships and feature
interactions that linear models cannot represent.

In this phase, we:
- Train tree-based classifiers on engineered features
- Compare performance against baseline models
- Analyze feature importance
- Perform targeted error analysis

These models form the primary results of the project.


### Feature Set for Tree-Based Models

Tree-based models are trained using engineered features only.
TF-IDF features are excluded to avoid excessive dimensionality and
to preserve interpretability.


In [49]:
# Engineered feature matrix (same as Phase 3)
engineered_features = [
    "password_length",
    "upper_count",
    "lower_count",
    "digit_count",
    "symbol_count",
    "upper_ratio",
    "digit_ratio",
    "symbol_ratio",
    "repetition_ratio",
    "entropy",
]

X_tree = pwlds_sample[engineered_features]
y_tree = pwlds_sample["strength"]

### Train / Validation Split

We reuse a stratified train-validation split to ensure fair comparison
with baseline models.


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_tree,
    y_tree,
    test_size=0.2,
    random_state=42,
    stratify=y_tree
)

### Random Forest Classifier

Random Forests capture non-linear interactions and are robust to feature
scaling. We limit hyperparameter tuning to avoid overfitting and keep
training time reasonable.


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score

In [52]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_leaf=20,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced"
)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_val)

print(classification_report(y_val, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12000
           1       0.93      0.94      0.94     12000
           2       0.87      0.93      0.90     12000
           3       0.92      0.84      0.88     12000
           4       1.00      1.00      1.00     12000

    accuracy                           0.94     60000
   macro avg       0.94      0.94      0.94     60000
weighted avg       0.94      0.94      0.94     60000



In [53]:
f1_rf = f1_score(y_val, y_pred_rf, average="macro")
f1_rf

0.9423091049717728

### Gradient Boosting Classifier

Gradient Boosting builds trees sequentially and often achieves higher
accuracy by correcting previous errors. We use conservative settings
to balance performance and training cost.


In [54]:
from sklearn.ensemble import GradientBoostingClassifier

In [55]:
gb_model = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_val)

print(classification_report(y_val, y_pred_gb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12000
           1       0.94      0.94      0.94     12000
           2       0.87      0.93      0.90     12000
           3       0.92      0.84      0.88     12000
           4       1.00      1.00      1.00     12000

    accuracy                           0.94     60000
   macro avg       0.94      0.94      0.94     60000
weighted avg       0.94      0.94      0.94     60000



In [56]:
f1_gb = f1_score(y_val, y_pred_gb, average="macro")
f1_gb

0.9430007047024429

### Tree-Based Model Comparison

We compare macro F1-scores across tree-based models to select the best
performing approach.


In [57]:
import pandas as pd

tree_results = pd.DataFrame({
    "Model": ["Random Forest", "Gradient Boosting"],
    "Macro_F1": [f1_rf, f1_gb]
})

tree_results

,Model,Macro_F1
0,Random Forest,0.942309
1,Gradient Boosting,0.943001


### Feature Importance Analysis

Feature importance helps interpret which password characteristics
contribute most to model decisions.


In [58]:
feature_importance = pd.DataFrame({
    "Feature": engineered_features,
    "Importance": rf_model.feature_importances_
}).sort_values(by="Importance", ascending=False)

feature_importance

,Feature,Importance
0,password_length,0.318270
4,symbol_count,0.174334
7,symbol_ratio,0.159903
2,lower_count,0.094574
1,upper_count,0.062790
5,upper_ratio,0.046064
6,digit_ratio,0.043212
9,entropy,0.038663
3,digit_count,0.035259
8,repetition_ratio,0.026931


### Error Analysis

We inspect misclassified passwords to understand systematic weaknesses
in the model and identify areas for improvement.


In [59]:
# Identify misclassified samples
errors = X_val.copy()
errors["true_strength"] = y_val.values
errors["pred_strength"] = y_pred_rf

misclassified = errors[errors["true_strength"] != errors["pred_strength"]]

misclassified.head()

,password_length,upper_count,lower_count,digit_count,symbol_count,upper_ratio,digit_ratio,symbol_ratio,repetition_ratio,entropy,true_strength,pred_strength
6073915,10,0,9,0,1,0.0,0.0,0.1,0.1,0.939794,3,2
4755885,10,2,7,0,1,0.2,0.0,0.1,0.0,1.000000,2,3
7639507,10,1,8,0,1,0.1,0.0,0.1,0.1,0.939794,3,2
6105721,10,0,9,0,1,0.0,0.0,0.1,0.1,0.939794,3,2
3307005,5,1,3,1,0,0.2,0.2,0.0,0.0,1.000000,1,0


### Error Analysis Observations

- Some long but repetitive passwords are overestimated
- Short passwords with symbols may still be misclassified
- These errors align with known limitations of rule-based and ML-based
  password strength estimation


## Phase 4 Summary

In this phase, we:
- Trained tree-based models on engineered features
- Captured non-linear feature interactions
- Compared Random Forest and Gradient Boosting performance
- Interpreted feature importance
- Performed targeted error analysis

The best-performing tree-based model serves as the primary model for
this project.


## ===============================
## Phase 5: External Dataset Comparison (Generalization Check)
## ===============================

### Purpose
Models that perform well on a single dataset may overfit dataset-specific
patterns. In this phase, we evaluate generalization by training and testing
a baseline model on a secondary password dataset.

This strengthens the credibility of the project by demonstrating how
results vary across datasets.


### Load Secondary Dataset

We load a publicly available password strength dataset (Kaggle) and use
it only for baseline comparison. No advanced feature engineering or
model tuning is performed on this dataset.


In [74]:
# Load secondary dataset
secondary = pd.read_csv(
    "secondary_data.csv",
    engine="python",
    on_bad_lines="skip"
)

secondary.head()

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1


### Data Quality Note

The external dataset contains malformed rows due to delimiter characters
inside password strings. We load the file using a tolerant parsing strategy
and skip corrupted rows, which has negligible impact on evaluation.

### Dataset Overview

We inspect the structure and label distribution of the secondary dataset
to understand differences relative to the primary dataset.


In [77]:
secondary.shape

(669640, 2)

In [65]:
secondary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669640 entries, 0 to 669639
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   password  669639 non-null  object
 1   strength  669640 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [66]:
secondary.isna().sum()

password    1
strength    0
dtype: int64

In [67]:
secondary = secondary.dropna(subset=["password"])
secondary.isna().sum()

password    0
strength    0
dtype: int64

### Column Standardization

To reuse feature engineering logic, we standardize column names and
ensure consistent data types.


In [68]:
# Rename columns if needed (adjust if column names differ)
secondary = secondary.rename(columns={
    "password": "password",
    "strength": "strength"
})

secondary["password"] = secondary["password"].astype(str)

### Feature Engineering (Minimal)

We apply the same structured feature engineering pipeline used in the
primary dataset, without adding TF-IDF or entropy-based enhancements.

In [70]:
# Password length
secondary["password_length"] = secondary["password"].str.len()

# Character counts
secondary["upper_count"] = secondary["password"].str.count(r"[A-Z]")
secondary["lower_count"] = secondary["password"].str.count(r"[a-z]")
secondary["digit_count"] = secondary["password"].str.count(r"[0-9]")
secondary["symbol_count"] = secondary["password"].str.count(r"[^A-Za-z0-9]")

# Ratios
secondary["upper_ratio"] = secondary["upper_count"] / secondary["password_length"]
secondary["digit_ratio"] = secondary["digit_count"] / secondary["password_length"]
secondary["symbol_ratio"] = secondary["symbol_count"] / secondary["password_length"]

### Train / Validation Split

We perform a simple stratified split for baseline evaluation.

In [71]:
from sklearn.model_selection import train_test_split

engineered_features = [
    "password_length",
    "upper_count",
    "lower_count",
    "digit_count",
    "symbol_count",
    "upper_ratio",
    "digit_ratio",
    "symbol_ratio"
]

X_sec = secondary[engineered_features]
y_sec = secondary["strength"]

X_sec_train, X_sec_val, y_sec_train, y_sec_val = train_test_split(
    X_sec,
    y_sec,
    test_size=0.2,
    random_state=42,
    stratify=y_sec
)

### Baseline Model on Secondary Dataset

We train a simple logistic regression model to establish a baseline
performance on the external dataset.

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

In [73]:
sec_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

sec_model.fit(X_sec_train, y_sec_train)

y_sec_pred = sec_model.predict(X_sec_val)

print(classification_report(y_sec_val, y_sec_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17940
           1       1.00      1.00      1.00     99360
           2       1.00      1.00      1.00     16628

    accuracy                           1.00    133928
   macro avg       1.00      1.00      1.00    133928
weighted avg       1.00      1.00      1.00    133928



In [75]:
f1_sec = f1_score(y_sec_val, y_sec_pred, average="macro")
f1_sec

0.9994740628874333

### Cross-Dataset Comparison

We compare baseline performance on the secondary dataset with results
from the primary dataset to highlight differences in data distribution
and labeling schemes.

In [76]:
comparison = pd.DataFrame({
    "Dataset": ["Primary (PWLDS)", "Secondary (Kaggle)"],
    "Model": ["Logistic Regression (Engineered)", "Logistic Regression (Engineered)"],
    "Macro_F1": [f1_eng, f1_sec]
})

comparison

,Dataset,Model,Macro_F1
0,Primary (PWLDS),Logistic Regression (Engineered),0.887627
1,Secondary (Kaggle),Logistic Regression (Engineered),0.999474


### Interpretation Note on External Dataset Performance

The near-perfect performance on the secondary (Kaggle) dataset reflects
simpler labeling rules and clearer class separation. In contrast, the
primary PWLDS dataset is larger, noisier, and more representative of
real-world password behavior, making it a more challenging and realistic
benchmark.

This comparison highlights the importance of evaluating password strength
models across multiple datasets rather than relying on a single metric.


## Phase 5 Summary

In this phase, we:
- Evaluated model performance on an external dataset
- Observed performance differences across datasets
- Demonstrated dataset dependency in password strength estimation
- Strengthened the generalization argument of the project

These results highlight the importance of dataset choice and motivate
careful evaluation when deploying password strength models in practice.

## ===============================
## Final Phase: Interpretation, Ethics & Project Conclusion
## ===============================

### Objective of the Final Phase
This phase consolidates results from all modeling stages, interprets them
from a security perspective, discusses limitations, and documents ethical
considerations. The goal is to present a complete, responsible, and
industry-aligned data science project.


## 1. Key Findings & Insights

### Feature Engineering
- Password length is the single most influential feature across all models.
- Character composition (symbols, digits, uppercase usage) significantly
  improves discrimination between medium and strong passwords.
- Ratios outperform raw counts, confirming that normalized features are
  more informative.
- Entropy provides complementary signal but is insufficient on its own,
  validating the need for hybrid feature sets.

### Modeling Results
- Linear models with engineered features perform strongly and are highly
  interpretable.
- Character-level TF-IDF captures sequence patterns missed by structured
  features.
- Combining engineered and NLP features yields the best overall performance.
- Tree-based models capture non-linear interactions and provide robust,
  interpretable results.


## 2. Model Comparison Summary

| Model Type | Strengths | Limitations |
|-----------|----------|-------------|
| Logistic Regression (Engineered) | Simple, interpretable, strong baseline | Limited non-linearity |
| Logistic Regression (TF-IDF) | Captures character patterns | Less interpretable |
| Logistic Regression (Combined) | Best overall performance | Higher complexity |
| Random Forest | Robust, interpretable | Slightly lower than combined linear |
| Gradient Boosting | Best non-linear model | Slower, harder to tune |

The Gradient Boosting model serves as the **primary interpretable non-linear
model**, while the combined Logistic Regression model achieves the highest
overall accuracy.

## 3. Security Interpretation

From a security standpoint:
- Weak passwords are reliably detected with very high recall, which is
  critical for enforcement systems.
- Errors primarily occur between adjacent strength classes, reflecting
  inherent ambiguity rather than model failure.
- Models align well with known password security principles:
  length, diversity, and unpredictability matter more than any single rule.

This confirms that machine learning–based approaches can meaningfully
augment traditional password strength heuristics.


## 4. External Dataset Generalization

Evaluation on an external Kaggle dataset yielded near-perfect performance
using simple engineered features.

This result reflects:
- Cleaner data
- Simpler or rule-based labeling
- Clearer class boundaries

In contrast, the primary PWLDS dataset is larger, noisier, and more
representative of real-world password behavior, making it a more realistic
benchmark. This comparison highlights the importance of cross-dataset
evaluation in security-related machine learning tasks.


## 5. Limitations

- Password strength labels are approximations and do not directly measure
  real-world attack cost.
- No simulated password-cracking attacks were performed.
- NLP features were trained on a sampled subset due to memory constraints.
- Results may vary across organizations with different password policies.

These limitations are common in password strength research and represent
opportunities for future improvement rather than flaws.


## 6. Ethical & Responsible Use Considerations

- No real user passwords were collected or logged during this project.
- All datasets used are publicly available and anonymized.
- The project does not provide password generation or cracking capabilities.
- Models are intended for **defensive security purposes only**, such as
  strength estimation and policy enforcement.

Any real-world deployment should:
- Avoid storing plaintext passwords
- Operate locally or on-device where possible
- Provide user feedback without logging sensitive inputs


## 7. Future Work

- Incorporate attack-based metrics (e.g., guess-number estimation)
- Explore probabilistic password models
- Integrate keyboard-walk and dictionary-word detection
- Deploy as a local API or browser-based strength checker
- Evaluate robustness against adversarial password construction


## Final Conclusion

This project demonstrates an end-to-end, production-aware approach to
password strength prediction using machine learning and NLP techniques.

By combining strong data preparation, interpretable feature engineering,
rigorous evaluation, and ethical considerations, the project provides a
realistic and defensible solution suitable for security-conscious
applications.

The methodology and results are representative of real-world data science
work in applied cybersecurity contexts.
